<a href="https://colab.research.google.com/github/marcivanmanalac/CNN_Preprocessor/blob/main/marcs_CNN_RESNET50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Google Drive.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check if the dataset is there

In [ ]:
!ls -l /content/drive/MyDrive/labeled_images_dataset

total 3740
-rw------- 1 root root 1901787 Feb 21 22:33 annotations.csv
drwx------ 2 root root    4096 Feb 20 06:57 East_Waiawa_Plots
drwx------ 2 root root    4096 Feb 20 06:57 East_Waiawa_Plots_Original
drwx------ 2 root root    4096 Feb 20 06:57 Loi_Behind_LLC
drwx------ 2 root root    4096 Feb 20 06:57 Loi_Behind_LLC_Original
-rw------- 1 root root  190286 Feb 21 22:33 test.csv
-rw------- 1 root root     178 Feb 21 22:34 test.gsheet
-rw------- 1 root root 1140763 Feb 21 22:33 train.csv
-rw------- 1 root root  570820 Feb 21 22:33 val.csv
drwx------ 2 root root    4096 Feb 20 06:57 West_Waiawa_Plots
drwx------ 2 root root    4096 Feb 20 06:57 West_Waiawa_Plots_Original


Run this Cell to update XML Paths for new labeled images


In [ ]:
import os
from xml.etree import ElementTree as ET

def batch_edit_xml(xml_directory, jpg_directory):
    count = 1 # initializing count to 1
    for root, dirs, files in os.walk(xml_directory):
        for file in files:
            if file.endswith(".xml"):
                file_path = os.path.join(root, file) # creating a file path by joining the root and the file name
                xml_tree = ET.parse(file_path) # parsing the XML file
                xml_root = xml_tree.getroot() # getting the root of the XML file
                filename = os.path.splitext(file)[0] # getting the file name without the extension
                jpg_path = os.path.join(jpg_directory, os.path.basename(root), filename + '.jpg').replace(os.sep, '/') # creating a jpg path and converting to forward slashes
                xml_root.find('./path').text = jpg_path # finding the path element in the XML file and updating it with the jpg_path
                xml_tree.write(file_path) # writing the changes back to the XML file
                print(f"{count} of {len(files)}: {file_path}") # printing the current count and the total number of files processed
                count += 1
                if count > len(files): # checking if the count has reached the length of the files
                    count = 1 # resetting the count back to 1
    print("Edit Complete") # indicating that the edit is complete

def main():
    ew_xml_dir = '/content/drive/MyDrive/labeled_images_dataset/East_Waiawa_Plots/East_Waiawa_Plots_XML/'
    ew_jpg_dir = '/content/drive/MyDrive/labeled_images_dataset/East_Waiawa_Plots/East_Waiawa_Plots_Cropped/'
    ww_xml_dir = '/content/drive/MyDrive/labeled_images_dataset/West_Waiawa_Plots/West_Waiawa_Plots_XML/'
    ww_jpg_dir = '/content/drive/MyDrive/labeled_images_dataset/West_Waiawa_Plots/West_Waiawa_Plots_Cropped/'
    loi_xml_dir = '/content/drive/MyDrive/labeled_images_dataset/Loi_Behind_LLC/Loi_Behind_LLC_XML/'
    loi_jpg_dir = '/content/drive/MyDrive/labeled_images_dataset/Loi_Behind_LLC/Loi_Behind_LLC_Cropped'
    batch_edit_xml(ew_xml_dir,ew_jpg_dir)
    batch_edit_xml(ww_xml_dir,ww_jpg_dir)
    batch_edit_xml(loi_xml_dir,loi_jpg_dir)


if __name__ == "__main__":
    main()



Create the Annotations CSV

In [ ]:
import argparse
import csv
import os
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET



def get_dirs():
    # Define XML Root Directory and CSV Output Directory
    xml_dir = '/content/drive/MyDrive/labeled_images_dataset/'
    csv_dir = '/content/drive/MyDrive/labeled_images_dataset/'
    class_names = input("Enter class names separated by commas: ").split(",")
    labels = input("Enter labels separated by commas: ").split(",")
    return xml_dir, csv_dir, class_names, labels


def process_xml_file(xml_file, class_names, labels):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    data = []
    for obj in root.findall("object"):
        name = obj.find("name").text
        if name in class_names:
            label = labels[class_names.index(name)].strip() # strip spaces from the label value
            bndbox = obj.find("bndbox")
            xmin = int(bndbox.find("xmin").text)
            ymin = int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = int(bndbox.find("ymax").text)
            path = root.find("path").text  # get the path tag
            #trying to fix CSV output
            #data.append((path, xmin, ymin, xmax, ymax, label))
            data.append((path, xmin, ymin, xmax, ymax, label))

    return data


def process_xml_files(xml_dir, csv_dir, class_names, labels):
    csv_file = os.path.join(csv_dir, "annotations.csv")
    with open(csv_file, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "path", "xmin", "ymin", "xmax", "ymax", "label"])  # add "path" column
        for root, dirs, files in os.walk(xml_dir):
            for file in files:
                if file.endswith(".xml"):
                    xml_file = os.path.join(root, file)
                    data = process_xml_file(xml_file, class_names, labels)
                    if data:
                        filename = os.path.splitext(file)[0] + ".jpg"
                        for row in data:
                            #EDIT
                            #writer.writerow(list(row))  # include path value as first element
                            writer.writerow([filename] + list(row))
def main():
    xml_dir, csv_dir, class_names, labels = get_dirs()
    process_xml_files(xml_dir, csv_dir, class_names, labels)
    print("CSV file saved to: ", os.path.join(csv_dir, "annotations.csv"))

if __name__ == "__main__":
    main()


Enter class names separated by commas: notch,infested_frond,infested_tree,healthy_frond,healthy_tree
Enter labels separated by commas: notch,infested_frond,infested_tree,healthy_frond,healthy_tree
CSV file saved to:  /content/drive/MyDrive/labeled_images_dataset/annotations.csv


Check the created CSV file

In [ ]:
import pandas as pd
csv_path = "/content/drive/MyDrive/labeled_images_dataset/annotations.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_path)
# Count the number of rows not including the header
num_rows = df.shape[0] - 1

# Print the number of rows
print("Lable Count: ", num_rows)
print(df.head(3))


Lable Count:  13529
  filename                                               path  xmin  ymin  \
0   32.jpg  /content/drive/MyDrive/labeled_images_dataset/...   605   134   
1   32.jpg  /content/drive/MyDrive/labeled_images_dataset/...   528    90   
2   32.jpg  /content/drive/MyDrive/labeled_images_dataset/...   511    66   

   xmax  ymax           label  
0   635   165           notch  
1   596   129           notch  
2   651   193  infested_frond  


Split the Annotations CSV to create training, validation, & test CSV files



In [ ]:
import csv
import random
import os

# Enter input CSV File Path and Destination
input_file_path = '/content/drive/MyDrive/labeled_images_dataset/annotations.csv'
output_dir_path = '/content/drive/MyDrive/labeled_images_dataset/'

# Path to the output CSV files for the training, validation, and testing sets
train_csv_file = os.path.join(output_dir_path, 'train.csv')
val_csv_file = os.path.join(output_dir_path, 'val.csv')
test_csv_file = os.path.join(output_dir_path, 'test.csv')

# Proportions of the split (train: 60%, val: 30%, test: 10%)
train_split = 0.6
val_split = 0.3
test_split = 0.1

# Read the CSV file and split the rows randomly
with open(input_file_path, 'r') as f:
    reader = csv.reader(f)
    header = next(reader, None)  # Skip the header row

    # Shuffle the rows randomly
    rows = list(reader)
    random.shuffle(rows)

    # Compute the number of rows for each split
    num_rows = len(rows)
    num_train = int(num_rows * train_split)
    num_val = int(num_rows * val_split)
    num_test = num_rows - num_train - num_val

    # Write the rows to the output CSV files
    with open(train_csv_file, 'w', newline='') as train_f, \
            open(val_csv_file, 'w', newline='') as val_f, \
            open(test_csv_file, 'w', newline='') as test_f:
        train_writer = csv.writer(train_f)
        val_writer = csv.writer(val_f)
        test_writer = csv.writer(test_f)

        # Write the rows to the output CSV files
        for i, row in enumerate(rows):
            if i < num_train:
                train_writer.writerow(row)
            elif i < num_train + num_val:
                val_writer.writerow(row)
            else:
                test_writer.writerow(row)
                
# Remove blank rows from the output CSV files
for file_path in [train_csv_file, val_csv_file, test_csv_file]:
    with open(file_path, 'r') as f:
        rows = list(csv.reader(f))
    
    with open(file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for row in rows:
            if any(row):
                writer.writerow(row)
                
print("Finished!")


Finished!


Check new train, val, and test CSV Files

 *** START HERE IF DATA IS READY ***

In [ ]:
import csv
import os
import pandas as pd

csv_train_path = "/content/drive/MyDrive/labeled_images_dataset/train.csv"  # Replace with the path to your CSV file
csv_val_path = "/content/drive/MyDrive/labeled_images_dataset/val.csv"  # Replace with the path to your CSV file
csv_test_path = "/content/drive/MyDrive/labeled_images_dataset/test.csv"  # Replace with the path to your CSV file

#Load Data Frame
df_train = pd.read_csv(csv_train_path)
df_val = pd.read_csv(csv_val_path)
df_test = pd.read_csv(csv_test_path)

#Print first three values
print(df_train.head(3))
print(df_val.head(3))
print(df_test.head(3))

  filename                                               path  xmin  ymin  \
0   11.jpg  /content/drive/MyDrive/labeled_images_dataset/...   740   520   
1   31.jpg  /content/drive/MyDrive/labeled_images_dataset/...   533   420   
2   33.jpg  /content/drive/MyDrive/labeled_images_dataset/...   126   369   

   xmax  ymax           label  
0   783   545           notch  
1   578   488   healthy_frond  
2   360   534  infested_frond  
  filename                                               path  xmin  ymin  \
0   35.jpg  /content/drive/MyDrive/labeled_images_dataset/...   348   591   
1   31.jpg  /content/drive/MyDrive/labeled_images_dataset/...   843   835   
2   24.jpg  /content/drive/MyDrive/labeled_images_dataset/...   750   104   

   xmax  ymax          label  
0   400   628          notch  
1  1000  1000  infested_tree  
2  1000   709   healthy_tree  
  filename                                               path  xmin  ymin  \
0   21.jpg  /content/drive/MyDrive/labeled_images_dat

Define:


* Preprocessing
* Data Augmentation
* Batch Size
* Image Size
* Generators




In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import warnings

# Define image preprocessing and data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size and image size
batch_size = 32
img_size = (224, 224)

# Load dataframes
df_train = pd.read_csv('train.csv')
df_val = pd.read_csv('val.csv')
df_test = pd.read_csv('test.csv')

# Define data generators for training, validation, and testing
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=df_val,
    x_col='path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Catch warning and extract invalid image filenames for train_generator
invalid_paths_train = []
with warnings.catch_warnings(record=True) as w:
    train_generator.next()
    for warning in w:
        if 'invalid image filename(s)' in str(warning.message):
            invalid_paths_train.extend(str(warning.message).split(' ')[-1].split(';')[:-1])
print('Invalid image paths for training generator:', invalid_paths_train)

# Catch warning and extract invalid image filenames for val_generator
invalid_paths_val = []
with warnings.catch_warnings(record=True) as w:
    val_generator.next()
    for warning in w:
        if 'invalid image filename(s)' in str(warning.message):
            invalid_paths_val.extend(str(warning.message).split(' ')[-1].split(';')[:-1])
print('Invalid image paths for validation generator:', invalid_paths_val)

# Catch warning and extract invalid image filenames for test_generator
invalid_paths_test = []
with warnings.catch_warnings(record=True) as w:
    test_generator.next()
    for warning in w:
        if 'invalid image filename(s)' in str(warning.message):
            invalid_paths_test.extend(str(warning.message).split(' ')[-1].split(';')[:-1])
print('Invalid image paths for test generator:', invalid_paths_test)


Found 6563 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 1555 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Found 3287 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 772 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Found 1098 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 255 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Define and Compile the model

In [ ]:
import os
import tensorflow as tf

# Ask user whether to use TPU or GPU
accelerator = input("Are you using a TPU or GPU runtime? Enter 'TPU' or 'GPU': ")

if accelerator == 'TPU':
  # TPU address
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  # Set the TPU address in the environment variable
  os.environ['TPU_NAME'] = tpu_address

  # Create a TPUClusterResolver object from the address
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)

  # Connect to the TPU
  tf.config.experimental_connect_to_cluster(resolver)

  # Initialize the TPU system
  tf.tpu.experimental.initialize_tpu_system(resolver)

  # Create the strategy using the resolver
  strategy = tf.distribute.TPUStrategy(resolver)
elif accelerator == 'GPU':
  # Create a MirroredStrategy for using the GPU
  strategy = tf.distribute.MirroredStrategy()
else:
  raise ValueError("Invalid accelerator. Must be 'TPU' or 'GPU'.")

# Define and compile the model within the strategy scope
with strategy.scope():
  model = tf.keras.models.Sequential([
      tf.keras.applications.ResNet50(
          include_top=False, weights='imagenet', input_shape=(224, 224, 3)
      ),
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dense(5, activation='softmax')
  ])
  model.compile(
      loss='categorical_crossentropy',
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=['accuracy']
  )



Are you using a TPU or GPU runtime? Enter 'TPU' or 'GPU': GPU
94765736/94765736 [==============================] - 4s 0us/step


Training steps

In [ ]:
# Define the number of training and validation steps per epoch
train_steps_per_epoch = train_generator.n // batch_size
val_steps_per_epoch = val_generator.n // batch_size

# Define the number of epochs for training
epochs = 10

Data Augmentation

In [ ]:
# Define data augmentation for training data
train_augmented_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2
)

Found 6563 validated image filenames belonging to 5 classes.


# **Fit model with augmentation**

In [ ]:
from tqdm import tqdm
# Fit the model with data augmentation
history = None
with tqdm(total=epochs, desc='Training') as pbar:
    for epoch in range(epochs):
        history = model.fit(
            train_augmented_generator,
            steps_per_epoch=train_steps_per_epoch,
            epochs=1,
            validation_data=val_generator,
            validation_steps=val_steps_per_epoch,
            verbose=0
        )
        pbar.update(1)

Training: 100%|██████████| 10/10 [1:00:29<00:00, 362.95s/it]


Save The model

In [ ]:
# Save the model architecture and weights
model.save('crb_model.h5')

In [ ]:
# Save the model architecture and weights
model.save('/content/drive/MyDrive/labeled_images_dataset/crb_model.h5')

**START HERE IF LOADING A MODEL**

In [ ]:
#load drive
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import tensorflow as tf
import keras
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import csv
import os
import pandas as pd

csv_test_path = "/content/drive/MyDrive/labeled_images_dataset/test.csv"  # Replace with the path to your CSV file

# Define batch size and image size
batch_size = 32
img_size = (224, 224)

#Load Data Frame
df_test = pd.read_csv(csv_test_path)

# Define image preprocessing and data augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1098 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 255 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(




# 1.   Evaluate model on test set
# 2.   NP Conversion
# 3.   Loss and Accuracy



In [4]:
import numpy as np
# Set random seed for reproducibility
np.random.seed(123)
model = load_model('/content/drive/MyDrive/labeled_images_dataset/crb_model.h5')
class_names = ['notch', 'infested_frond', 'infested_tree', 'healthy_frond', 'healthy_tree']

# Evaluate the model on the test set
test_generator.reset()
y_pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
print(f'model.predict: {y_pred}')
y_true = test_generator.classes

# Convert predictions from probabilities to class indices
y_pred = np.argmax(y_pred, axis=1)

print(f'Probabilities to class indices array: {y_pred}')

print(f'Shape of y_pred: {y_pred.shape}')

# Evaluate the model on the test set
test_generator.reset()
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=2)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


35/35 [==============================] - 22s 587ms/step
model.predict: [[1.0774378e-02 4.2675366e-04 3.2745188e-01 8.5703939e-02 5.7564300e-01]
 [9.4519460e-01 5.2036025e-02 5.1819981e-04 6.2199874e-04 1.6291504e-03]
 [7.1029305e-01 4.9777780e-02 8.8216037e-02 2.9705910e-02 1.2200716e-01]
 ...
 [9.2529327e-01 7.4056908e-02 8.8848567e-05 1.6270435e-04 3.9822311e-04]
 [8.3612114e-01 5.1163107e-02 3.5673600e-02 1.8076336e-02 5.8965772e-02]
 [8.6499131e-01 7.8981072e-02 1.7045557e-02 7.9624103e-03 3.1019645e-02]]
Probabilities to class indices array: [4 0 0 ... 0 0 0]
Shape of y_pred: (1098,)
35/35 - 21s - loss: 1.7775 - accuracy: 0.6129 - 21s/epoch - 586ms/step
Test Loss: 1.7774949073791504
Test Accuracy: 0.6129326224327087


Generate classification report and confusion matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
class_names = ['notch', 'infested_frond', 'infested_tree', 'healthy_frond', 'healthy_tree']

# Reshape y_pred
y_pred_reshaped = y_pred[:, np.newaxis]

# Generate the classification report and confusion matrix
report = classification_report(y_true, np.argmax(y_pred_reshaped, axis=1), target_names=class_names)
print(report)

cm = confusion_matrix(y_true, np.argmax(y_pred_reshaped, axis=1))
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(class_names)
ax.yaxis.set_ticklabels(class_names)

AxisError: ignored

In [ ]:
from tqdm import tqdm
# Generate the AUC-ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
num_classes = 5 # notch, infested_frond, infested_tree, healthy_frond, healthy_tree
for i in tqdm(range(num_classes), desc='Computing ROC curve'):
    fpr[i], tpr[i], _ = roc_curve(y_true == i, y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

Computing ROC curve:   0%|          | 0/5 [00:00<?, ?it/s]


ValueError: ignored

In [ ]:
# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(label_binarize(y_true, classes=range(num_classes)).ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Plot ROC curve for each class
plt.figure()
lw = 2
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])
for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw, label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
    
plt.plot(fpr["micro"], tpr["micro"], color='deeppink', lw=lw, linestyle='--',
         label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]))

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
